In [1]:
#| default_exp convert_to_protobuf
%load_ext autoreload
%autoreload 2

In [2]:
#| include: false
from nbdev.showdoc import *

In [3]:
#| export
from feems.components_model import Engine, Switchboard, ShaftLine, \
    MainEngineForMechanicalPropulsion, MechanicalPropulsionComponent, \
    MainEngineWithGearBoxForMechanicalPropulsion
from typing import cast, Union, List

from feems.types_for_feems import TypeComponent, NOxCalculationMethod, EmissionCurve
from feems.components_model.component_mechanical import EngineDualFuel
from feems.components_model.component_electric import (
    ElectricComponent,
    FuelCellSystem,
    ElectricMachine,
    BatterySystem,
    Battery,
    SuperCapacitorSystem,
    PTIPTO,
    SuperCapacitor
)
from feems.components_model.component_electric import SerialSystemElectric, Genset
from feems.system_model import ElectricPowerSystem, MechanicalPropulsionSystem, \
    MechanicalPropulsionSystemWithElectricPowerSystem, HybridPropulsionSystem

import MachSysS.system_structure_pb2 as proto


def convert_efficiency_curve_to_protobuf(component: ElectricComponent) -> proto.Efficiency:
    """Convert efficiency value or curve in the component to protobuf message"""
    efficiency = proto.Efficiency()
    if len(component._efficiency_points) == 1:
        efficiency.value = component._efficiency_points[0]
    else:
        efficiency.curve.curve.points.extend([
            proto.Point(x=each_point[0], y=each_point[1])
            for each_point in component._efficiency_points
        ])
        efficiency.curve.x_label = "power load"
        efficiency.curve.y_label = "efficiency"
    return efficiency


def convert_bsfc_curve_to_protobuf(component: Union[Engine, EngineDualFuel], for_pilot_fuel: bool = False) -> proto.BSFC:
    """Convert bsfc value or curve in the component to protobuf message"""
    bsfc = proto.BSFC()
    bsfc_points = component.specific_fuel_consumption_points if not for_pilot_fuel else component.specific_pilot_fuel_consumption_points
    if len(bsfc_points) == 1:
        bsfc.value = bsfc_points[0]
    else:
        bsfc.curve.curve.points.extend([
            proto.Point(x=each_point[0], y=each_point[1]) for each_point in bsfc_points
        ])
        bsfc.curve.x_label = "power load"
        bsfc.curve.y_label = "bsfc"
    return bsfc


def convert_electric_machine_to_protobuf(
    component: ElectricMachine,
    order_from_switchboard: int = 1
) -> proto.ElectricMachine:
    """Convert elecrtic machine component of FEEMS to protobuf message"""
    return proto.ElectricMachine(
        name=component.name,
        rated_power_kw=component.rated_power,
        rated_speed_rpm=component.rated_speed,
        efficiency=convert_efficiency_curve_to_protobuf(component),
        order_from_switchboard_or_shaftline=order_from_switchboard
    )


def convert_electric_component_to_protobuf(
    component: ElectricComponent,
    order_from_switchboard: int = 1
) -> proto.ElectricMachine:
    """Convert converter component of FEEMS to protobuf message"""
    return proto.ElectricComponent(
        name=component.name,
        rated_power_kw=component.rated_power,
        efficiency=convert_efficiency_curve_to_protobuf(component),
        order_from_switchboard_or_shaftline=order_from_switchboard
    )


def convert_battery_component_to_protobuf(
    component: Battery,
    order_from_switchboard: int = 1
) -> proto.ElectricMachine:
    """Convert battery component of FEEMS to protobuf message"""
    return proto.Battery(
        name=component.name,
        energy_capacity_kwh=component.rated_capacity_kWh,
        rated_charging_rate_c=component.charging_rate_C,
        rated_discharging_rate_c=component.discharging_rate_C,
        efficiency_charging=component.eff_charging,
        efficiency_discharging=component.eff_discharging,
        initial_state_of_charge=component.soc0,
        order_from_switchboard_or_shaftline=order_from_switchboard
    )


def convert_supercapacitor_component_to_protobuf(
    component: SuperCapacitor,
    order_from_switchboard: int = 1
) -> proto.ElectricMachine:
    """Convert converter component of FEEMS to protobuf message"""
    return proto.SuperCapacitor(
        name=component.name,
        energy_capacity_wh=component.rated_capacity_Wh,
        rated_power_kw=component.rated_power,
        efficiency_charging=component.eff_charging,
        efficiency_discharging=component.eff_discharging,
        initial_state_of_charge=component.soc0,
        order_from_switchboard_or_shaftline=order_from_switchboard
    )


def convert_serial_electric_system_to_protobuf(
    component: Union[SerialSystemElectric, PTIPTO],
    initial_order_from_switchboard: int = 1
) -> proto.Subsystem:
    """Convert serial electric system or PTI/PTO component to protobuf message"""
    order = initial_order_from_switchboard
    subsystem = proto.Subsystem()
    for subcomponent in component.components:
        if subcomponent.type == TypeComponent.TRANSFORMER:
            subsystem.transformer.CopyFrom(convert_electric_component_to_protobuf(
                component=subcomponent, order_from_switchboard=order
            ))
        if subcomponent.type in [TypeComponent.POWER_CONVERTER, TypeComponent.INVERTER, TypeComponent.RECTIFIER, TypeComponent.ACTIVE_FRONT_END]:
            if not subsystem.HasField('converter1'):
                subsystem.converter1.CopyFrom(convert_electric_component_to_protobuf(
                    component=subcomponent, order_from_switchboard=order
                ))
            else:
                subsystem.converter2.CopyFrom(convert_electric_component_to_protobuf(
                    component=subcomponent, order_from_switchboard=order
                ))
        if subcomponent.type in [
            TypeComponent.SYNCHRONOUS_MACHINE,
            TypeComponent.INDUCTION_MACHINE,
            TypeComponent.ELECTRIC_MOTOR
        ]:
            subsystem.electric_machine.CopyFrom(convert_electric_machine_to_protobuf(
                component=subcomponent, order_from_switchboard=order
            ))
        order += 1
    return subsystem


def convert_nox_calculation_method_to_protobuf(nox_calculation_method_feems: NOxCalculationMethod) -> proto.Engine.NOxCalculationMethod:
    """Convert nox calculation method of FEEMS to protobuf message"""
    index = proto.Engine.NOxCalculationMethod.Value(nox_calculation_method_feems.name)
    return index


def convert_emission_curves_to_protobuf(emission_curves_feems: List[EmissionCurve]) -> List[proto.EmissionCurve]:
    """Convert emission curves of FEEMS to protobuf message"""
    if emission_curves_feems is None:
        return []
    return [
        proto.EmissionCurve(
            x_label="load_ratio",
            y_label="emission_g_per_kwh",
            curve=proto.Curve1D(
                points=[
                    proto.Point(x=each_point.load_ratio, y=each_point.emission_g_per_kwh)
                    for each_point in each_curve.points_per_kwh
                ]
            ),
            emission_type=proto.EmissionType.Value(each_curve.emission_type.value)
        )
        for each_curve in emission_curves_feems
    ]


def convert_engine_component_to_protobuf(
    engine_feems: Union[Engine, EngineDualFuel],
    order_from_shaftline_or_switchboard: int = 1
) -> proto.Engine:
    """Convert engine component of FEEMS to protobuf message"""
    engine = proto.Engine(
        name=engine_feems.name,
        rated_power_kw=engine_feems.rated_power,
        rated_speed_rpm=engine_feems.rated_speed,
        bsfc=convert_bsfc_curve_to_protobuf(engine_feems),
        main_fuel=proto.Fuel(
            fuel_type=engine_feems.fuel_type.value,
            fuel_origin=engine_feems.fuel_origin.value,
        ),
        nox_calculation_method=convert_nox_calculation_method_to_protobuf(engine_feems.nox_calculation_method),
        emission_curves = convert_emission_curves_to_protobuf(engine_feems.emission_curves),
        order_from_switchboard_or_shaftline=order_from_shaftline_or_switchboard
    )
    if isinstance(engine_feems, EngineDualFuel):
        engine.pilot_bsfc.CopyFrom(convert_bsfc_curve_to_protobuf(engine_feems, for_pilot_fuel=True))
        engine.pilot_fuel.CopyFrom(proto.Fuel(
            fuel_type=engine_feems.pilot_fuel_type.value,
            fuel_origin=engine_feems.pilot_fuel_origin.value,
        ))
    return engine


def convert_switchboard_to_protobuf(switchboard_feems: Switchboard) -> proto.Switchboard:
    switchboard_proto = proto.Switchboard()
    switchboard_proto.switchboard_id = switchboard_feems.id

    for component in switchboard_feems.components:
        subsystem = proto.Subsystem(
            power_type=component.power_type.value,
            component_type=component.type.value,
            name=component.name,
            rated_power_kw=component.rated_power,
            rated_speed_rpm=component.rated_speed
        )
        if component.type == TypeComponent.GENERATOR:
            subsystem.electric_machine.CopyFrom(convert_electric_machine_to_protobuf(
                component=component,
            ))
        elif component.type == TypeComponent.FUEL_CELL_SYSTEM:
            component = cast(FuelCellSystem, component)
            subsystem.converter1.CopyFrom(convert_electric_component_to_protobuf(
                component=component.converter
            ))
            subsystem.fuel_cell.CopyFrom(proto.FuelCell(
                name=component.fuel_cell.name,
                rated_power_kw=component.fuel_cell.rated_power,
                efficiency=convert_efficiency_curve_to_protobuf(component.fuel_cell),
                fuel=proto.Fuel(
                    fuel_type=component.fuel_cell.fuel_type.value,
                    fuel_origin=component.fuel_cell.fuel_origin.value,
                ),
                number_modules=component.number_modules,
                order_from_switchboard_or_shaftline=2
            ))
        elif component.type == TypeComponent.GENSET:
            component = cast(Genset, component)
            subsystem.electric_machine.CopyFrom(convert_electric_machine_to_protobuf(
                component=component.generator,
            ))
            subsystem.engine.CopyFrom(convert_engine_component_to_protobuf(
                engine_feems=component.aux_engine,
                order_from_shaftline_or_switchboard=2
            ))
        elif component.type == TypeComponent.OTHER_LOAD:
            subsystem.other_load.CopyFrom(
                convert_electric_component_to_protobuf(component=component))
        elif component.type in [TypeComponent.PTI_PTO_SYSTEM, TypeComponent.PROPULSION_DRIVE]:
            subsystem.MergeFrom(convert_serial_electric_system_to_protobuf(component=component))
        elif component.type == TypeComponent.BATTERY_SYSTEM:
            component = cast(BatterySystem, component)
            subsystem.converter1.CopyFrom(convert_electric_component_to_protobuf(component=component.converter))
            subsystem.battery.CopyFrom(convert_battery_component_to_protobuf(
                component=component.battery,
                order_from_switchboard=2
            ))
        elif component.type == TypeComponent.BATTERY:
            subsystem.battery.CopyFrom(
                convert_battery_component_to_protobuf(component=component))
        elif component.type == TypeComponent.SUPERCAPACITOR_SYSTEM:
            component = cast(SuperCapacitorSystem, component)
            subsystem.converter1.CopyFrom(convert_electric_component_to_protobuf(component=component.converter))
            subsystem.battery.CopyFrom(convert_supercapacitor_component_to_protobuf(
                component=component.supercapacitor,
                order_from_switchboard=2
            ))
        elif component.type == TypeComponent.SUPERCAPACITOR:
            subsystem.battery.CopyFrom(
                convert_supercapacitor_component_to_protobuf(component=component))
        else:
            raise TypeError(
                f"The component type ({component.type.name}) is not a proper type for an electric "
                f"system or the conversion for the type is not implemented.")
        switchboard_proto.subsystems.append(subsystem)
    return switchboard_proto


def convert_shaftline_to_protobuf(shaftline_feems: ShaftLine) -> proto.ShaftLine:
    """Convert shaft line to protobuf message"""
    shaftline_proto = proto.ShaftLine()
    shaftline_proto.shaft_line_id = shaftline_feems.id
    gear_proto = None
    propeller_id = 1
    # If there is a gear box component on the shaft line, it is recognized as a gear box connecting
    # the main engine to the propeller and pti/pto.
    # The gear box is added to the protobuf message first.
    for gear in filter(
        lambda component: component.type == TypeComponent.GEARBOX,
        shaftline_feems.components
    ):
        gear = cast(MechanicalPropulsionComponent, gear)
        gear_proto = proto.Gear(
            name=gear.name,
            rated_power_kw=gear.rated_power,
            rated_speed_rpm=gear.rated_speed,
            efficiency=convert_efficiency_curve_to_protobuf(gear),
            order_from_switchboard_or_shaftline=1,
        )

    for component in shaftline_feems.components:
        subsystem = proto.Subsystem(
            power_type=component.power_type.value,
            component_type=component.type.value,
            name=component.name,
            rated_power_kw=component.rated_power,
            rated_speed_rpm=component.rated_speed
        )
        if component.type == TypeComponent.MAIN_ENGINE:
            component = cast(MainEngineForMechanicalPropulsion, component)
            subsystem.engine.CopyFrom(convert_engine_component_to_protobuf(
                engine_feems=component.engine,
                order_from_shaftline_or_switchboard=1
            ))
        elif component.type == TypeComponent.MAIN_ENGINE_WITH_GEARBOX:
            if gear_proto is not None:
                raise ValueError(
                    f"The shaft line {shaftline_feems.id} already has a gear box. "
                    f"Please use a main-engine component rather than a main-engine-with-a-gear-box"
                    f"component."
                )
            component = cast(MainEngineWithGearBoxForMechanicalPropulsion, component)
            subsystem.engine.CopyFrom(convert_engine_component_to_protobuf(
                engine_feems=component.main_engine,
                order_from_shaftline_or_switchboard=2
            ))
            subsystem.gear.CopyFrom(proto.Gear(
                name=component.gearbox.name,
                rated_power_kw=component.gearbox.rated_power,
                rated_speed_rpm=component.gearbox.rated_speed,
                efficiency=convert_efficiency_curve_to_protobuf(component.gearbox),
                order_from_switchboard_or_shaftline=1,
            ))
        elif component.type == TypeComponent.PROPELLER_LOAD:
            if gear_proto is not None:
                subsystem.gear.CopyFrom(gear_proto)
            subsystem.propeller.CopyFrom(proto.Propeller(
                efficiency=convert_efficiency_curve_to_protobuf(component),
                propulsor_id=propeller_id,
                order_from_switchboard_or_shaftline=2,
            ))
            propeller_id += 1
        elif component.type == TypeComponent.PTI_PTO_SYSTEM:
            subsystem.MergeFrom(convert_serial_electric_system_to_protobuf(component=component))
        elif component.type == TypeComponent.GEARBOX:
            continue
        else:
            raise ValueError(f"The shaftline contains a component ({component.name}) that has an "
                             f"imcompatible type ({component.type}) for conversion.")
        shaftline_proto.subsystems.append(subsystem)
    return shaftline_proto


In [4]:
# Test converting
from tests.utility import create_switchboard_with_components, create_shaftline_with_components
from MachSysS.convert_to_protobuf import convert_switchboard_to_protobuf, convert_shaftline_to_protobuf

switchboard_feems = create_switchboard_with_components(
    switchboard_id=1,
    rated_power_available_total=10000,
    no_power_sources=2,
    no_power_consumer=3
)

switchboard = convert_switchboard_to_protobuf(switchboard_feems)
assert len(switchboard_feems.components) == len(switchboard.subsystems)
for subsystem, component_feems in zip(switchboard.subsystems, switchboard_feems.components):
    assert subsystem.component_type == component_feems.type.value

shaftline_feems = create_shaftline_with_components(
    shaft_line_id=1,
    rated_power_available_total=10000,
    no_power_sources=2,
    no_power_consumer=1,
    has_gear=True,
    has_pti_pto=True
)

shaftline_proto = convert_shaftline_to_protobuf(shaftline_feems)
print(shaftline_proto)

shaft_line_id: 1
subsystems {
  engine {
    name: "Main engine 1"
    rated_power_kw: 5859.0016556055216
    rated_speed_rpm: 994.47618733206048
    bsfc {
      curve {
        x_label: "power load"
        y_label: "bsfc"
        curve {
          points {
            x: 0.1
            y: 19.319933023361479
          }
          points {
            x: 0.2
            y: 112.16731626254233
          }
          points {
            x: 0.30000000000000004
            y: 23.099504235318278
          }
          points {
            x: 0.4
            y: 60.638884895192049
          }
          points {
            x: 0.5
            y: 48.678858766795628
          }
          points {
            x: 0.6
            y: 77.979773772621641
          }
          points {
            x: 0.70000000000000007
            y: 33.609600342009728
          }
          points {
            x: 0.8
            y: 47.867957805787853
          }
          points {
            x: 0.9
            y: 29

In [5]:
#| export
def convert_electric_system_to_protobuf(electric_system: ElectricPowerSystem) \
        -> proto.ElectricSystem:
    """Convert electric system to protobuf message"""
    return proto.ElectricSystem(
        switchboards=[
            convert_switchboard_to_protobuf(switchboard)
            for _, switchboard in electric_system.switchboards.items()
        ]
    )


def convert_electric_system_to_protobuf_machinery_system(
    electric_system: ElectricPowerSystem,
    maximum_allowed_genset_load_percentage: float = 80.0
) -> proto.MachinerySystem:
    """Convert electric system to protobuf message as a machinery system"""
    return proto.MachinerySystem(
        name=electric_system.name,
        propulsion_type=proto.MachinerySystem.PropulsionType.ELECTRIC,
        fuel_storage=[],
        maximum_allowed_genset_load_percentage=maximum_allowed_genset_load_percentage,
        electric_system=convert_electric_system_to_protobuf(electric_system)
    )

def convert_mechanical_system_to_protobuf(
    mechanical_propulsion_system: MechanicalPropulsionSystem
) -> proto.MechanicalSystem:
    return proto.MechanicalSystem(
        shaft_lines=[
            convert_shaftline_to_protobuf(shaftline)
            for shaftline in mechanical_propulsion_system.shaft_line
        ]
    )


def convert_mechanical_propulsion_system_with_electric_system_to_protobuf(
    system_feems: MechanicalPropulsionSystemWithElectricPowerSystem,
    maximum_allowed_genset_load_percentage: float = 80.0,
) -> proto.MachinerySystem:
    return proto.MachinerySystem(
        name=system_feems.name,
        propulsion_type=proto.MachinerySystem.PropulsionType.MECHANICAL,
        fuel_storage=[],
        maximum_allowed_genset_load_percentage=maximum_allowed_genset_load_percentage,
        mechanical_system=convert_mechanical_system_to_protobuf(system_feems.mechanical_system),
        electric_system=convert_electric_system_to_protobuf(system_feems.electric_system)
    )


def convert_hybrid_propulsion_system_to_protobuf(
    system_feems: HybridPropulsionSystem,
    maximum_allowed_genset_load_percentage: float = 80.0,
) -> proto.MachinerySystem:
    return proto.MachinerySystem(
        name=system_feems.name,
        propulsion_type=proto.MachinerySystem.HYBRID,
        fuel_storage=[],
        maximum_allowed_genset_load_percentage=maximum_allowed_genset_load_percentage,
        mechanical_system=convert_mechanical_system_to_protobuf(system_feems.mechanical_system),
        electric_system=convert_electric_system_to_protobuf(system_feems.electric_system)
    )

## Test converting to protobuf for electric system

In [6]:
import random
import os

import numpy as np
from MachSysS.convert_to_protobuf import (
    convert_electric_system_to_protobuf, 
    convert_mechanical_system_to_protobuf, 
    convert_mechanical_propulsion_system_with_electric_system_to_protobuf,
    convert_electric_system_to_protobuf_machinery_system
)
from feems.system_model import (
    ElectricPowerSystem,
    MechanicalPropulsionSystem,
    MechanicalPropulsionSystemWithElectricPowerSystem
)
from feems.components_model.component_mechanical import (
    Engine
)
from feems.components_model.component_electric import (
    Genset,
    Battery,
    ElectricComponent,
    ElectricMachine,
    SerialSystemElectric,
    FuelCellSystem,
    FuelCell,
    BatterySystem,
)
from feems.fuel import (
    TypeFuel, FuelOrigin
)
from feems.types_for_feems import (
    TypePower, TypeComponent, NOxCalculationMethod
)
from tests.utility import create_switchboard_with_components, create_shaftline_with_components

components = []
for switchboard_id in [1,2,3]:
    components.extend(create_switchboard_with_components(
        switchboard_id=switchboard_id,
        rated_power_available_total=10000*random.random(),
        no_power_sources=2,
        no_power_consumer=3
    ).components)
electric_system = ElectricPowerSystem(
    name="El 1",
    power_plant_components=components,
    bus_tie_connections=[(1,2), (2,3)]
)
electric_system_proto = convert_electric_system_to_protobuf(electric_system)
assert len(electric_system.switchboards) == len(electric_system_proto.switchboards)

mechanical_system = MechanicalPropulsionSystem(
    name="Me 1",
    components_list=create_shaftline_with_components(
        shaft_line_id=1,
        rated_power_available_total=10000,
        no_power_consumer=1,
        no_power_sources=1,
    ).components,
)
mechanical_system_proto = convert_mechanical_system_to_protobuf(mechanical_system)
assert len(mechanical_system_proto.shaft_lines) == len(mechanical_system.shaft_line)
system_feems = MechanicalPropulsionSystemWithElectricPowerSystem(
    name="ME Propulsion System",
    mechanical_system=mechanical_system,
    electric_system=electric_system
)
system_proto = convert_mechanical_propulsion_system_with_electric_system_to_protobuf(system_feems)
print(system_proto)

# Create an electric propulsion system
number_gensets = 4
gensets = []
for index in range(number_gensets):
    gensets.append(Genset(
        name=f"Genset {index + 1}",
        aux_engine=Engine(
            type_=TypeComponent.AUXILIARY_ENGINE,
            rated_power=1000,
            rated_speed=1000,
            bsfc_curve=np.array([
                [0.1, 270], [0.25, 240], [0.5, 220], [0.75, 210], [1.0, 220]
            ]),
            nox_calculation_method=NOxCalculationMethod.TIER_3,
            fuel_type=TypeFuel.DIESEL,
            fuel_origin=FuelOrigin.FOSSIL,
        ),
        generator=ElectricMachine(
            type_=TypeComponent.GENERATOR,
            power_type=TypePower.POWER_SOURCE,
            name=f"Generator {index + 1}",
            rated_power=1000,
            rated_speed=1000,
            eff_curve=np.array([
                [0.1, 0.70], [0.25, 0.85], [0.5, 0.96], [0.75, 0.97],
            ]),
            switchboard_id=int(index/2) + 1
        ),
    ))
energy_storages = []
power_consumers = []
fuel_cell_systems = []
number_batteries = 2
for index in range(number_batteries):
    energy_storages.append(BatterySystem(
        name="Battery system 1",
        battery=Battery(
            name="Battery 1",
            rated_capacity_kwh=1000,
            charging_rate_c=1,
            discharge_rate_c=1,
            switchboard_id=index + 1
        ),
        converter=ElectricComponent(
            type_=TypeComponent.POWER_CONVERTER,
            power_type=TypePower.POWER_TRANSMISSION,
            name="Converter 1",
            rated_power=1000,
            eff_curve=np.array([
                [0.1, 0.90], [0.25, 0.95], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]
            ]),
        ),
        switchboard_id=index + 1
    ))
    
number_propulsion_drives = 2
for index in range(number_propulsion_drives):
    power_consumers.append(SerialSystemElectric(
        name="Propulsion Drive 1",
        type_=TypeComponent.PROPULSION_DRIVE,
        power_type=TypePower.POWER_CONSUMER,
        components=[
            ElectricComponent(
                type_=TypeComponent.POWER_CONVERTER,
                power_type=TypePower.POWER_TRANSMISSION,
                name="Converter 1",
                rated_power=2000,
                eff_curve=np.array([
                    [0.1, 0.90], [0.25, 0.95], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]
                ]),
                switchboard_id=index + 1
            ),
            ElectricMachine(
                type_=TypeComponent.SYNCHRONOUS_MACHINE,
                power_type=TypePower.POWER_CONSUMER,
                name="Propulsion motor 1",
                rated_power=2000,
                rated_speed=1000,
                eff_curve=np.array([
                    [0.1, 0.70], [0.25, 0.85], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]
                ]),
                switchboard_id=index + 1
            ),
        ],
        switchboard_id=index + 1,
        rated_power=2000,
        rated_speed=1000,
    ))

# Add other load
power_consumers.append(ElectricComponent(
    type_=TypeComponent.OTHER_LOAD,
    power_type=TypePower.POWER_CONSUMER,
    name="Other load 1",
    rated_power=1000,
    switchboard_id=1
))    

# Add fuel cells
number_fuel_cells = 2
for index in range(number_fuel_cells):
    fuel_cell_systems.append(FuelCellSystem(
        name="Fuel cell system 1",
        fuel_cell_module=FuelCell(
            name="Fuel cell 1",
            rated_power=1000,
            eff_curve=np.array([
                [0.1, 0.70], [0.25, 0.85], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]
            ]),
            fuel_type=TypeFuel.HYDROGEN,
            fuel_origin=FuelOrigin.RENEWABLE_NON_BIO,
        ),
        converter=ElectricComponent(
            type_=TypeComponent.POWER_CONVERTER,
            power_type=TypePower.POWER_TRANSMISSION,
            name="Converter 1",
            rated_power=3150,
            eff_curve=np.array([
                [0.1, 0.90], [0.25, 0.95], [0.5, 0.96], [0.75, 0.97], [1.0, 0.96]
            ]),
        ),
        switchboard_id=index + 1,
        number_modules=3
    ))
# Create the electric system
electric_system = ElectricPowerSystem(
    name="Electric propulsion system",
    power_plant_components=gensets + energy_storages + power_consumers + fuel_cell_systems,
    bus_tie_connections=[(1,2)]
)

system_proto = convert_electric_system_to_protobuf_machinery_system(electric_system)
fuel_cells = [
    subsystem.fuel_cell for switchboard in system_proto.electric_system.switchboards
    for subsystem in switchboard.subsystems
    if subsystem.HasField("fuel_cell")
]
for each_fuel_cell in fuel_cells:
    assert each_fuel_cell.number_modules == 3

with open(os.path.join("tests", "system_proto.mss"), "wb") as file:
    file.write(system_proto.SerializeToString())

2023-11-27 15:04:20,468 - tests.utility - WARNING - Efficiency of generator is not supplied, using random monotonic curve
2023-11-27 15:04:20,468 - tests.utility - WARNING - Efficiency of engine is not supplied, using random monotonic curve
2023-11-27 15:04:20,482 - tests.utility - WARNING - Efficiency of generator is not supplied, using random monotonic curve
2023-11-27 15:04:20,483 - tests.utility - WARNING - Efficiency of engine is not supplied, using random monotonic curve
2023-11-27 15:04:20,485 - tests.utility - WARNING - Efficiency of generator is not supplied, using random monotonic curve
2023-11-27 15:04:20,485 - tests.utility - WARNING - Efficiency of engine is not supplied, using random monotonic curve


name: "ME Propulsion System"
maximum_allowed_genset_load_percentage: 80
mechanical_system {
  shaft_lines {
    shaft_line_id: 1
    subsystems {
      engine {
        name: "Main engine 1"
        rated_power_kw: 10000
        rated_speed_rpm: 952.98559560399542
        bsfc {
          curve {
            x_label: "power load"
            y_label: "bsfc"
            curve {
              points {
                x: 0.1
                y: 41.572729151082008
              }
              points {
                x: 0.2
                y: 105.88714628085762
              }
              points {
                x: 0.30000000000000004
                y: 150.11478926479333
              }
              points {
                x: 0.4
                y: 197.5465115992514
              }
              points {
                x: 0.5
                y: 67.932055130502533
              }
              points {
                x: 0.6
                y: 142.10722200377813
              }
     

## Test converting to protobuf for mechanical system

In [7]:
# Create a system
import os
import numpy as np
from feems.components_model.component_mechanical import (
    Engine,
    EngineDualFuel,
    MechanicalPropulsionComponent,
    MainEngineForMechanicalPropulsion
)
from feems.components_model.component_electric import (
    Genset,
    ElectricComponent, ElectricMachine,
)
from feems.system_model import (
    MechanicalPropulsionSystem,
    ElectricPowerSystem,
    MechanicalPropulsionSystemWithElectricPowerSystem
)
from feems.types_for_feems import (
    TypeComponent, Power_kW, Speed_rpm, TypePower
)
from feems.fuel import TypeFuel, FuelOrigin
from MachSysS.convert_to_protobuf import convert_mechanical_propulsion_system_with_electric_system_to_protobuf
from functools import reduce

# Create the engine
bsfc_for_main_engine = np.array([
    [.25, 151.2],
    [.30, 149.5],
    [.40, 146.3],
    [.50, 143.2],
    [.60, 141.7],
    [.70, 141.0],
    [.75, 140.8],
    [.80, 140.9],
    [.85, 141.0],
    [.90, 141.4],
    [.95, 141.9],
    [1.00, 142.8]
])
bspfc_for_main_engine = np.array([
    [.25, 1.6],
    [.30, 1.4],
    [.40, 1.1],
    [.50, 0.9],
    [.60, 0.8],
    [.70, 0.8],
    [.75, 0.7],
    [.80, 0.7],
    [.85, 0.7],
    [.90, 0.6],
    [.95, 0.6],
    [1.00, 0.6]
])
main_engine = MainEngineForMechanicalPropulsion(
    engine=EngineDualFuel(
        type_=TypeComponent.MAIN_ENGINE,
        name="Main engine",
        rated_power=Power_kW(18000),
        rated_speed=Speed_rpm(60),
        bsfc_curve=bsfc_for_main_engine,
        fuel_type=TypeFuel.NATURAL_GAS,
        fuel_origin=FuelOrigin.FOSSIL,
        bspfc_curve=bspfc_for_main_engine,
        pilot_fuel_type=TypeFuel.DIESEL,
        nox_calculation_method=NOxCalculationMethod.TIER_3,
    ),
    name="Main engine for mechanical propulsion",
    shaft_line_id=1
)
propeller = MechanicalPropulsionComponent(
    type_=TypeComponent.PROPELLER_LOAD,
    power_type=TypePower.POWER_CONSUMER,
    name="Propeller",
    rated_power=Power_kW(25000),
    rated_speed=Speed_rpm(60),
    shaft_line_id=1
)
mechanical_propulsion_system = MechanicalPropulsionSystem(
    name="Mechanical propulsion system",
    components_list=[main_engine, propeller]
)
number_gensets = 3
bsfc_for_aux_engine = np.array([
    [.25, 248.0],
    [.30, 235.0],
    [.35, 225.0],
    [.40, 218.0],
    [.45, 213.0],
    [.50, 208.0],
    [.55, 205.0],
    [.60, 203.0],
    [.65, 200.0],
    [.70, 199.0],
    [.75, 197.0],
    [.80, 197.0],
    [.85, 196.0],
    [.90, 196.0],
    [.95, 196.0],
    [1.00, 195.2]
])
gensets = [
    Genset(
        name=f"Genset {i + 1}",
        aux_engine=Engine(
            type_=TypeComponent.AUXILIARY_ENGINE,
            name=f"Aux engine {i + 1}",
            rated_power=Power_kW(1290),
            rated_speed=Speed_rpm(900),
            bsfc_curve=bsfc_for_aux_engine,
            fuel_type=TypeFuel.NATURAL_GAS,
            fuel_origin=FuelOrigin.FOSSIL,
            nox_calculation_method=NOxCalculationMethod.TIER_3,
        ),
        generator=ElectricMachine(
            type_=TypeComponent.GENERATOR,
            name=f"Generator {i + 1}",
            rated_power=Power_kW(1000),
            rated_speed=Speed_rpm(900),
            eff_curve=np.array([
                [0.0, 0.88],
                [0.1, 0.89],
                [0.2, 0.90],
                [0.3, 0.91],
                [0.4, 0.92],
                [0.5, 0.93],
                [0.6, 0.93],
                [0.7, 0.94],
                [0.8, 0.94],
                [0.9, 0.95],
                [1.0, 0.95],
            ]),
            power_type=TypePower.POWER_SOURCE,
            switchboard_id=int(i/2) + 1
        ),
    )
    for i in range(number_gensets)
]
other_load = ElectricComponent(
    type_=TypeComponent.OTHER_LOAD,
    name="Other load",
    rated_power=Power_kW(4000),
    power_type=TypePower.POWER_CONSUMER,
    switchboard_id=1
)
electric_power_system = ElectricPowerSystem(
    name="Electric power system",
    power_plant_components=[*gensets, other_load],
    bus_tie_connections=[(1, 1)]
)
machinery_system = MechanicalPropulsionSystemWithElectricPowerSystem(
    name="Base case system",
    mechanical_system=mechanical_propulsion_system,
    electric_system=electric_power_system
)

In [8]:
# Conversion of the mechanical system
mechanical_system_proto = convert_mechanical_system_to_protobuf(
    mechanical_propulsion_system=mechanical_propulsion_system
)
print(mechanical_system_proto)

shaft_lines {
  shaft_line_id: 1
  subsystems {
    engine {
      name: "Main engine"
      rated_power_kw: 18000
      rated_speed_rpm: 60
      bsfc {
        curve {
          x_label: "power load"
          y_label: "bsfc"
          curve {
            points {
              x: 0.25
              y: 151.2
            }
            points {
              x: 0.3
              y: 149.5
            }
            points {
              x: 0.4
              y: 146.3
            }
            points {
              x: 0.5
              y: 143.2
            }
            points {
              x: 0.6
              y: 141.7
            }
            points {
              x: 0.7
              y: 141
            }
            points {
              x: 0.75
              y: 140.8
            }
            points {
              x: 0.8
              y: 140.9
            }
            points {
              x: 0.85
              y: 141
            }
            points {
              x: 0.9
  

In [9]:
# Conversion of the mechanical propulsion system with electric power system
mechanical_propulsion_system_with_electric_system_proto = convert_mechanical_propulsion_system_with_electric_system_to_protobuf(
    system_feems=machinery_system
)
print(mechanical_propulsion_system_with_electric_system_proto)

name: "Base case system"
maximum_allowed_genset_load_percentage: 80
mechanical_system {
  shaft_lines {
    shaft_line_id: 1
    subsystems {
      engine {
        name: "Main engine"
        rated_power_kw: 18000
        rated_speed_rpm: 60
        bsfc {
          curve {
            x_label: "power load"
            y_label: "bsfc"
            curve {
              points {
                x: 0.25
                y: 151.2
              }
              points {
                x: 0.3
                y: 149.5
              }
              points {
                x: 0.4
                y: 146.3
              }
              points {
                x: 0.5
                y: 143.2
              }
              points {
                x: 0.6
                y: 141.7
              }
              points {
                x: 0.7
                y: 141
              }
              points {
                x: 0.75
                y: 140.8
              }
              points {
          

In [10]:
# Conversion of the hybrid propulsion system
# Create a pto system
switchboard_id = 1
rated_power = 1000
synch_mach = ElectricMachine(
    type_=TypeComponent.SYNCHRONOUS_MACHINE,
    power_type=TypePower.PTI_PTO,
    name="synchronous machine",
    rated_power=rated_power,
    rated_speed=1000,
    switchboard_id=switchboard_id,
)

# Create a rectifier instance
rectifier = ElectricComponent(
    type_=TypeComponent.RECTIFIER,
    power_type=TypePower.POWER_TRANSMISSION,
    name="rectifier",
    rated_power=rated_power,
    eff_curve=np.array([99.5]),
    switchboard_id=switchboard_id
)

# Create a inverter instance
inverter = ElectricComponent(
    type_=TypeComponent.INVERTER,
    power_type=TypePower.POWER_TRANSMISSION,
    name="inverter",
    rated_power=rated_power,
    eff_curve=np.array([98.5]),
    switchboard_id=switchboard_id
)

# Create a transformer instance
transformer = ElectricComponent(
    type_=TypeComponent.TRANSFORMER,
    power_type=TypePower.POWER_TRANSMISSION,
    name="transformer",
    rated_power=rated_power,
    eff_curve=np.array([99]),
    switchboard_id=switchboard_id
)

pto = PTIPTO(
    name="PTI PTO",
    components=[synch_mach, rectifier, inverter, transformer],
    rated_power=1000,
    rated_speed=1000,
    switchboard_id=synch_mach.switchboard_id,
    shaft_line_id=1,
)

# Creat a new electric system with pto
all_electric_components = reduce(
    lambda acc, switchboard: [*acc, *switchboard.components],
    machinery_system.electric_system.switchboards.values(),
    [pto]
)
bus_tie_breakers = reduce(
    lambda acc, bus_tie_breaker: [*acc, bus_tie_breaker.switchboard_ids],
    machinery_system.electric_system.bus_tie_breakers,
    []
)
electric_system_with_pto = ElectricPowerSystem(
    name="electric power system with PTO",
    power_plant_components=all_electric_components,
    bus_tie_connections=bus_tie_breakers,
)

# Create a new mechanical system with pto
all_mechanical_components = reduce(
    lambda acc, shaftline: [*acc, *shaftline.components],
    machinery_system.mechanical_system.shaft_line,
    [pto]
)

mechanical_system_with_pto = MechanicalPropulsionSystem(
    name="mechanical system with PTO",
    components_list=all_mechanical_components,
)

# Create a hybrid system
hybrid_prop_system_feems = HybridPropulsionSystem(
    name="Hybrid system",
    electric_system=electric_system_with_pto,
    mechanical_system=mechanical_system_with_pto,
)
hybrid_prop_system_proto = convert_hybrid_propulsion_system_to_protobuf(
    system_feems=hybrid_prop_system_feems
)
print(hybrid_prop_system_proto)

name: "Hybrid system"
propulsion_type: HYBRID
maximum_allowed_genset_load_percentage: 80
mechanical_system {
  shaft_lines {
    shaft_line_id: 1
    subsystems {
      electric_machine {
        name: "synchronous machine"
        rated_power_kw: 1000
        rated_speed_rpm: 1000
        efficiency {
          curve {
            x_label: "power load"
            y_label: "efficiency"
            curve {
              points {
                y: 1
              }
              points {
                x: 1
                y: 1
              }
            }
          }
        }
        order_from_switchboard_or_shaftline: 1
      }
      transformer {
        name: "transformer"
        rated_power_kw: 1000
        efficiency {
          curve {
            x_label: "power load"
            y_label: "efficiency"
            curve {
              points {
                y: 99
              }
              points {
                x: 1
                y: 99
              }
           

In [11]:
# Save the systems to a file
electric_system_with_pto_proto = convert_electric_system_to_protobuf(
    electric_system=electric_system_with_pto
)
mechanical_system_with_pto_proto = convert_mechanical_system_to_protobuf(
    mechanical_propulsion_system=mechanical_system_with_pto
)
electric_propulsion_system_proto = convert_electric_system_to_protobuf_machinery_system(
    electric_system=electric_system,
    maximum_allowed_genset_load_percentage=80
)
files_to_save = {
    "electric_system_with_pto.mss": electric_system_with_pto_proto,
    "mechanical_system_with_pto.mss": mechanical_system_with_pto_proto,
    "electric_propulsion_system.mss": electric_propulsion_system_proto,
    "mechanical_propulsion_with_electric_system.mss": mechanical_propulsion_system_with_electric_system_proto,
    "hybrid_propulsion_system.mss": hybrid_prop_system_proto,
}
for file_name, proto_message in files_to_save.items():
    with open(os.path.join("tests", file_name), "wb") as f:
        f.write(proto_message.SerializeToString())